In [7]:
from contextlib import contextmanager
import sqlite3
import yaml

In [8]:
curr_dir = ! pwd
curr_dir = curr_dir[0]
print(curr_dir)
assert curr_dir.endswith("hyperborea3/scripts/spells")

/home/jeremy/projects/hyperborea3/scripts/spells


In [9]:
load_folders = ["lvl1", "lvl2", "lvl3", "lvl4", "lvl5", "lvl6"]

In [10]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [11]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the <i>acid arrow</i> strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul><li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li></ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. 

In [12]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [13]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [52, 53, 75, 105, 346]:
        assert spell["Desc"][-4:] == "</p>"



In [14]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

In [15]:
for folder in load_folders:
    files = ! ls {folder}/*.yml
    for f in files:
        spell = load_yml_spell(f)
        spell_preload_check(spell)
        update_spell(spell)
